In [28]:
import pandas as pd
import os
import sqlalchemy
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine
import numpy as np

In [29]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

# Switch depending on which table you want to read.
# sql_query = 'select * from resale_prices_based_on_registration_date;'
sql_query = 'select * from resale_prices_based_on_approval_date;'

with engine.connect() as cnxn:
    approval = pd.read_sql(sql_query, cnxn)


approval

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,n_rooms,storey_range_min,storey_range_max
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,NaN,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,NaN,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,NaN,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,NaN,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313689,2012-02-01,GEYLANG,4 ROOM,319,UBI AVE 1,10 TO 12,84.0,Simplified,1985,383000.0,NaN,10.0,12.0
1313690,2012-02-01,GEYLANG,4 ROOM,314,UBI AVE 1,01 TO 03,84.0,Simplified,1985,382000.0,NaN,1.0,3.0
1313691,2012-02-01,GEYLANG,4 ROOM,343,UBI AVE 1,01 TO 03,84.0,Simplified,1986,380000.0,NaN,1.0,3.0
1313692,2012-02-01,HOUGANG,3 ROOM,631,HOUGANG AVE 8,07 TO 09,60.0,Improved,1985,300000.0,NaN,7.0,9.0


In [30]:
def clean_approval_df(df):

    df['month'] = pd.to_datetime(df['month'])
    df['n_rooms'] = df['flat_type'].str.extract('(\d)').astype('Int64')
    df['n_rooms'] = df['n_rooms'].astype('Int64')
    # Convert story range to min and max int columns
    df['storey_range_min'] = df['storey_range'].str[:2].astype(int)
    df['storey_range_max'] = df['storey_range'].str[-2:].astype(int)

    df.sort_values('month', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

approval = clean_approval_df(approval)
approval

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,n_rooms,storey_range_min,storey_range_max
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,1,10,12
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,5,1,3
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,5,1,3
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,5,7,9
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,5,10,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313689,2012-02-01,SENGKANG,4 ROOM,268A,COMPASSVALE LINK,16 TO 18,90.0,Premium Apartment,2006,525000.0,4,16,18
1313690,2012-02-01,SENGKANG,4 ROOM,268D,COMPASSVALE LINK,13 TO 15,90.0,Premium Apartment,2006,535000.0,4,13,15
1313691,2012-02-01,SENGKANG,4 ROOM,269D,COMPASSVALE LINK,16 TO 18,90.0,Premium Apartment,2006,563000.0,4,16,18
1313692,2012-02-01,SENGKANG,4 ROOM,184C,RIVERVALE CRES,16 TO 18,86.0,Model A2,2003,423000.0,4,16,18


In [42]:
# engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)
#
# # Switch depending on which table you want to read.
# sql_query = 'select * from resale_prices_based_on_registration_date;'
# # sql_query = 'select * from resale_prices_based_on_approval_date;'
#
# with engine.connect() as cnxn:
#     registration = pd.read_sql(sql_query, cnxn)


In [64]:
def create_resale_registration():

    base_dir = '../data/resale-flat-prices/'
    df = pd.DataFrame()
    for file in os.listdir('../data/resale-flat-prices/'):
        if "registration-date" in file and 'csv' in file:
            df = pd.concat([df, pd.read_csv(f'{base_dir}{file}')])

    return df

registration = create_resale_registration()
registration

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
37148,2016-12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,82
37149,2016-12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,69
37150,2016-12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,74
37151,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,70


In [65]:
def clean_registration_df(df):

    df['month'] = pd.to_datetime(df['month'])
    df['n_rooms'] = df['flat_type'].str.extract('(\d)').astype('Int64')
    df['n_rooms'] = df['n_rooms'].astype('Int64')
    # Convert story range to min and max int columns
    df['storey_range_min'] = df['storey_range'].str[:2].astype(int)
    df['storey_range_max'] = df['storey_range'].str[-2:].astype(int)

    # Convert remaining lease years to int years and int months remaining columns
    df['remaining_lease_years'] = df['remaining_lease'].str.extract('(?P<remaining_lease_years>\d{2}) year[s]?')

    df[['remaining_lease_months', 'remaining_lease_month']] = df['remaining_lease'].str.extract('\d{2} year[s]? (?P<remaining_lease_months>\d{2}) month[s]?|(?P<remaining_lease_month>\d{2}) month[s]?')
    df.drop('remaining_lease_month', axis=1, inplace=True)
    df[['remaining_lease_years', 'remaining_lease_months']] = df[['remaining_lease_years', 'remaining_lease_months']].astype('Int64').copy()
    df['remaining_lease_years'].fillna(df['remaining_lease'], inplace=True)
    df.sort_values('month', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)
    df['remaining_lease_months'].fillna(pd.Series(np.where(df['remaining_lease'].isnull(), np.nan, 0)), inplace=True)



    return df

In [66]:
registration = clean_registration_df(registration)
registration

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,n_rooms,storey_range_min,storey_range_max,remaining_lease_years,remaining_lease_months
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN,2,6,10,<NA>,<NA>
1,2012-03-01,SENGKANG,4 ROOM,184B,RIVERVALE CRES,11 TO 15,90.0,Model A,2003,436000.0,NaN,4,11,15,<NA>,<NA>
2,2012-03-01,SENGKANG,4 ROOM,201B,COMPASSVALE DR,11 TO 15,90.0,Model A,2001,437000.0,NaN,4,11,15,<NA>,<NA>
3,2012-03-01,SENGKANG,4 ROOM,311B,ANCHORVALE LANE,01 TO 05,95.0,Premium Apartment,2002,438000.0,NaN,4,1,5,<NA>,<NA>
4,2012-03-01,SENGKANG,4 ROOM,259B,COMPASSVALE RD,01 TO 05,85.0,Model A2,2001,438000.0,NaN,4,1,5,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,CLEMENTI,3 ROOM,208B,CLEMENTI AVE 6,19 TO 21,68.0,Model A,2017,575000.0,94 years 08 months,3,19,21,94,8
210826,2022-03-01,CLEMENTI,3 ROOM,320,CLEMENTI AVE 4,04 TO 06,67.0,New Generation,1979,380888.0,56 years 02 months,3,4,6,56,2
210827,2022-03-01,CLEMENTI,3 ROOM,334,CLEMENTI AVE 2,07 TO 09,67.0,New Generation,1978,416000.0,55 years 05 months,3,7,9,55,5
210828,2022-03-01,GEYLANG,3 ROOM,3,JOO CHIAT RD,10 TO 12,74.0,Model A,1985,470000.0,62 years 10 months,3,10,12,62,10


In [83]:
def create_resale_prices(approval_df, registration_df):
    rs_df = pd.concat([approval_df, registration_df])

    rs_df['month'] = pd.to_datetime(rs_df['month'])
    rs_df['lease_commence_date'] = pd.to_datetime(rs_df['lease_commence_date'], format='%Y')

    rs_df['id'] = rs_df['month'].dt.strftime('%Y') + "_" + rs_df['month'].dt.strftime('%m') + "_" +  rs_df['block'] + "_" + rs_df['street_name'].str.replace(' ', '_') + "_" + rs_df['storey_range'].str.replace(' ', '_')

    # Remaining lease years =
    # if month >= 6
    # lease_commence_date + 99 - month
    # if month < 6
    # lease_commence_date + 98 - month
    rs_df['remaining_lease_years'] = np.where(rs_df['month'].dt.month >= 6, ((rs_df['lease_commence_date'].dt.year + 98) - rs_df['month'].dt.year), ((rs_df['lease_commence_date'].dt.year + 99) - rs_df['month'].dt.year))

    # Convert back to int year.
    rs_df['lease_commence_date'] = rs_df['lease_commence_date'].dt.year.astype(int)


    # Price per square foot and meter
    rs_df['price_per_sq_ft'] = rs_df['resale_price'] / (rs_df['floor_area_sqm'] * 3.28084) #3.28084 ft per meter
    rs_df['price_per_sq_m'] = rs_df['resale_price'] / rs_df['floor_area_sqm']

    # Price per square foot/meter and remaining lease years
    rs_df['price_per_sq_ft_per_lease_yr'] = rs_df['price_per_sq_ft'] / rs_df['remaining_lease_years']
    rs_df['price_per_sq_m_per_lease_yr'] = rs_df['price_per_sq_m'] / rs_df['remaining_lease_years']



    return rs_df

rs_df = create_resale_prices(approval, registration)
rs_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,storey_range_min,storey_range_max,remaining_lease,remaining_lease_years,remaining_lease_months,id,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,10,12,NaN,86,<NA>,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,88.490320,290.322581,1.028957,3.375844
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,...,1,3,NaN,94,<NA>,1990_01_229_YISHUN_ST_21_01_TO_03,248.626108,815.702479,2.644959,8.677686
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,...,1,3,NaN,94,<NA>,1990_01_231_YISHUN_ST_21_01_TO_03,236.786769,776.859504,2.519008,8.264463
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,...,7,9,NaN,94,<NA>,1990_01_221_YISHUN_ST_21_07_TO_09,246.862802,809.917355,2.626200,8.616142
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,...,10,12,NaN,94,<NA>,1990_01_161_YISHUN_ST_11_10_TO_12,302.280982,991.735537,3.215755,10.550378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,CLEMENTI,3 ROOM,208B,CLEMENTI AVE 6,19 TO 21,68.0,Model A,2017,575000.0,...,19,21,94 years 08 months,94,8,2022_03_208B_CLEMENTI_AVE_6_19_TO_21,2577.352859,8455.882353,27.418647,89.956195
210826,2022-03-01,CLEMENTI,3 ROOM,320,CLEMENTI AVE 4,04 TO 06,67.0,New Generation,1979,380888.0,...,4,6,56 years 02 months,56,2,2022_03_320_CLEMENTI_AVE_4_04_TO_06,1732.756100,5684.895522,30.942073,101.515991
210827,2022-03-01,CLEMENTI,3 ROOM,334,CLEMENTI AVE 2,07 TO 09,67.0,New Generation,1978,416000.0,...,7,9,55 years 05 months,55,5,2022_03_334_CLEMENTI_AVE_2_07_TO_09,1892.489492,6208.955224,34.408900,112.890095
210828,2022-03-01,GEYLANG,3 ROOM,3,JOO CHIAT RD,10 TO 12,74.0,Model A,1985,470000.0,...,10,12,62 years 10 months,62,10,2022_03_3_JOO_CHIAT_RD_10_TO_12,1935.891830,6351.351351,31.224062,102.441151


In [84]:
def update_street_abbreviations(resale_df):
    resale_df['address']=resale_df['block']+' '+resale_df['street_name']

    ##Deal with short forms
    resale_df['address']=resale_df['address'].str.replace(' AVE ',' AVENUE ')
    resale_df['address']=resale_df['address'].str.replace(' ST ',' STREET ')
    resale_df['address']=resale_df['address'].str.replace(' NTH',' NORTH')
    resale_df['address']=resale_df['address'].str.replace(' STH',' SOUTH')
    resale_df['address']=resale_df['address'].str.replace(' RD',' ROAD')
    resale_df['address']=resale_df['address'].str.replace(' UPP ',' UPPER ')
    resale_df['address']=resale_df['address'].str.replace(' CTRL',' CENTRAL')
    resale_df['address']=resale_df['address'].str.replace(' JLN ',' JALAN ')
    resale_df['address']=resale_df['address'].str.replace('BT ','BUKIT ')
    resale_df['address']=resale_df['address'].str.replace(' HTS',' HEIGHTS')
    resale_df['address']=resale_df['address'].str.replace(' MKT ',' MARKET ')
    resale_df['address']=resale_df['address'].str.replace('TG ','TANJONG ')
    resale_df['address']=resale_df['address'].str.replace("C'WEALTH",'COMMONWEALTH')
    resale_df['address']=resale_df['address'].str.replace("LOR ",'LORONG ')
    resale_df['address']=resale_df['address'].str.replace("GDNS",'GARDENS')
    resale_df['address']=resale_df['address'].str.replace('ST. ','SAINT ')
    resale_df['address']=resale_df['address'].str.replace(' PK',' PARK')
    resale_df['address']=resale_df['address'].str.replace(' DR ',' DRIVE ')
    resale_df['address']=resale_df['address'].str.replace('KG ','KAMPONG ')
    resale_df['address']=resale_df['address'].str.replace(' CL ',' CLOSE ')

    return resale_df

In [85]:
rs_df = update_street_abbreviations(rs_df)
rs_df

/var/folders/l5/fj2kg32d6rz92n5r1t_xxk880000gn/T/ipykernel_83372/2010800487.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  resale_df['address']=resale_df['address'].str.replace('ST. ','SAINT ')


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,storey_range_max,remaining_lease,remaining_lease_years,remaining_lease_months,id,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr,address
0,1990-01-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,...,12,NaN,86,<NA>,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,88.490320,290.322581,1.028957,3.375844,309 ANG MO KIO AVENUE 1
1,1990-01-01,YISHUN,5 ROOM,229,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,98700.0,...,3,NaN,94,<NA>,1990_01_229_YISHUN_ST_21_01_TO_03,248.626108,815.702479,2.644959,8.677686,229 YISHUN STREET 21
2,1990-01-01,YISHUN,5 ROOM,231,YISHUN ST 21,01 TO 03,121.0,IMPROVED,1985,94000.0,...,3,NaN,94,<NA>,1990_01_231_YISHUN_ST_21_01_TO_03,236.786769,776.859504,2.519008,8.264463,231 YISHUN STREET 21
3,1990-01-01,YISHUN,5 ROOM,221,YISHUN ST 21,07 TO 09,121.0,IMPROVED,1985,98000.0,...,9,NaN,94,<NA>,1990_01_221_YISHUN_ST_21_07_TO_09,246.862802,809.917355,2.626200,8.616142,221 YISHUN STREET 21
4,1990-01-01,YISHUN,5 ROOM,161,YISHUN ST 11,10 TO 12,121.0,IMPROVED,1985,120000.0,...,12,NaN,94,<NA>,1990_01_161_YISHUN_ST_11_10_TO_12,302.280982,991.735537,3.215755,10.550378,161 YISHUN STREET 11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210825,2022-03-01,CLEMENTI,3 ROOM,208B,CLEMENTI AVE 6,19 TO 21,68.0,Model A,2017,575000.0,...,21,94 years 08 months,94,8,2022_03_208B_CLEMENTI_AVE_6_19_TO_21,2577.352859,8455.882353,27.418647,89.956195,208B CLEMENTI AVENUE 6
210826,2022-03-01,CLEMENTI,3 ROOM,320,CLEMENTI AVE 4,04 TO 06,67.0,New Generation,1979,380888.0,...,6,56 years 02 months,56,2,2022_03_320_CLEMENTI_AVE_4_04_TO_06,1732.756100,5684.895522,30.942073,101.515991,320 CLEMENTI AVENUE 4
210827,2022-03-01,CLEMENTI,3 ROOM,334,CLEMENTI AVE 2,07 TO 09,67.0,New Generation,1978,416000.0,...,9,55 years 05 months,55,5,2022_03_334_CLEMENTI_AVE_2_07_TO_09,1892.489492,6208.955224,34.408900,112.890095,334 CLEMENTI AVENUE 2
210828,2022-03-01,GEYLANG,3 ROOM,3,JOO CHIAT RD,10 TO 12,74.0,Model A,1985,470000.0,...,12,62 years 10 months,62,10,2022_03_3_JOO_CHIAT_RD_10_TO_12,1935.891830,6351.351351,31.224062,102.441151,3 JOO CHIAT ROAD


In [86]:
def sort_cols(resale_df):
    resale_df = resale_df[['id', 'month', 'block', 'town', 'street_name', 'address', 'storey_range', 'storey_range_min', 'storey_range_max',
      'flat_model', 'flat_type', 'n_rooms', 'floor_area_sqm',
       'lease_commence_date', 'remaining_lease', 'remaining_lease_years', 'remaining_lease_months',
      'resale_price', 'price_per_sq_ft', 'price_per_sq_m', 'price_per_sq_ft_per_lease_yr', 'price_per_sq_m_per_lease_yr']].sort_values('month', ascending=True)

    return resale_df

In [87]:
rs_df = sort_cols(rs_df)
rs_df

,id,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr
0,1990_01_309_ANG_MO_KIO_AVE_1_10_TO_12,1990-01-01,309,ANG MO KIO,ANG MO KIO AVE 1,309 ANG MO KIO AVENUE 1,10 TO 12,10,12,IMPROVED,...,31.0,1977,NaN,86,<NA>,9000.0,88.490320,290.322581,1.028957,3.375844
2004,1990_01_307_CLEMENTI_AVE_4_01_TO_03,1990-01-01,307,CLEMENTI,CLEMENTI AVE 4,307 CLEMENTI AVENUE 4,01 TO 03,1,3,NEW GENERATION,...,67.0,1979,NaN,88,<NA>,47000.0,213.814919,701.492537,2.429715,7.971506
2005,1990_01_309_CLEMENTI_AVE_4_04_TO_06,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,82.0,1980,NaN,89,<NA>,57000.0,211.873164,695.121951,2.380597,7.810359
2006,1990_01_309_CLEMENTI_AVE_4_10_TO_12,1990-01-01,309,CLEMENTI,CLEMENTI AVE 4,309 CLEMENTI AVENUE 4,10 TO 12,10,12,NEW GENERATION,...,67.0,1980,NaN,89,<NA>,34000.0,154.674622,507.462687,1.737917,5.701828
2007,1990_01_310_CLEMENTI_AVE_4_04_TO_06,1990-01-01,310,CLEMENTI,CLEMENTI AVE 4,310 CLEMENTI AVENUE 4,04 TO 06,4,6,NEW GENERATION,...,67.0,1980,NaN,89,<NA>,47000.0,213.814919,701.492537,2.402415,7.881939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210529,2022_03_501B_YISHUN_ST_51_07_TO_09,2022-03-01,501B,YISHUN,YISHUN ST 51,501B YISHUN STREET 51,07 TO 09,7,9,Improved,...,112.0,2018,94 years 11 months,95,11,652000.0,1774.371372,5821.428571,18.677593,61.278195
210528,2022_03_820_YISHUN_ST_81_07_TO_09,2022-03-01,820,YISHUN,YISHUN ST 81,820 YISHUN STREET 81,07 TO 09,7,9,Improved,...,122.0,1988,65 years 06 months,65,6,585000.0,1461.540937,4795.081967,22.485245,73.770492
210527,2022_03_359_YISHUN_RING_RD_01_TO_03,2022-03-01,359,YISHUN,YISHUN RING RD,359 YISHUN RING ROAD,01 TO 03,1,3,Apartment,...,145.0,1988,65 years 05 months,65,5,682000.0,1433.610989,4703.448276,22.055554,72.360743
210538,2022_03_751_YISHUN_ST_72_04_TO_06,2022-03-01,751,YISHUN,YISHUN ST 72,751 YISHUN STREET 72,04 TO 06,4,6,New Generation,...,91.0,1984,61 years 02 months,61,2,458000.0,1534.048303,5032.967033,25.148333,82.507656


In [78]:
rs_df.dtypes

month                           datetime64[ns]
block                                   object
town                                    object
street_name                             object
address                                 object
storey_range                            object
storey_range_min                         int64
storey_range_max                         int64
flat_model                              object
flat_type                               object
n_rooms                                  Int64
floor_area_sqm                         float64
lease_commence_date                      int64
remaining_lease                         object
remaining_lease_years                    int64
remaining_lease_months                   Int64
resale_price                           float64
price_per_sq_ft                        float64
price_per_sq_m                         float64
price_per_sq_ft_per_lease_yr           float64
price_per_sq_m_per_lease_yr            float64
dtype: object

In [82]:
rs_df['month'].dt.strftime('%Y') + "_" + rs_df['month'].dt.strftime('%m') + "_" +  rs_df['block'] + "_" + rs_df['street_name'].str.replace(' ', '_') + "_" + rs_df['storey_range'].str.replace(' ', '_')

0         1990_01309_ANG_MO_KIO_AVE_1_10_TO_12
1             1990_01229_YISHUN_ST_21_01_TO_03
2             1990_01231_YISHUN_ST_21_01_TO_03
3             1990_01221_YISHUN_ST_21_07_TO_09
4             1990_01161_YISHUN_ST_11_10_TO_12
                          ...                 
210825     2022_03208B_CLEMENTI_AVE_6_19_TO_21
210826      2022_03320_CLEMENTI_AVE_4_04_TO_06
210827      2022_03334_CLEMENTI_AVE_2_07_TO_09
210817           2022_0340_CIRCUIT_RD_10_TO_12
210829            2022_03149_PETIR_RD_01_TO_03
Length: 1524524, dtype: object

In [75]:
rs_df[rs_df['remaining_lease_years'].isnull() == True]

,month,block,town,street_name,address,storey_range,storey_range_min,storey_range_max,flat_model,flat_type,...,floor_area_sqm,lease_commence_date,remaining_lease,remaining_lease_years,remaining_lease_months,resale_price,price_per_sq_ft,price_per_sq_m,price_per_sq_ft_per_lease_yr,price_per_sq_m_per_lease_yr


In [97]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_alc_cnxn = engine.connect()

rs_df.to_sql('resale_prices', sql_alc_cnxn, if_exists='replace', index=False)

sql_alc_cnxn.close()